# ¿Cómo medir rendimiento y riesgo en un portafolio? II

<img style="float: right; margin: 0px 0px 15px 15px;" src="http://www.picpedia.org/clipboard/images/stock-portfolio.jpg" width="600px" height="400px" />

> La clase pasada y la presente, están dedicadas a obtener medidas de rendimiento y riesgo en un portafolio.

> Vimos que podemos obtener los rendimientos de un portafolio mediante la relación

$r_p=\sum_{i=1}^{n}w_ir_i$

, y una vez teniendo los rendimientos del portafolio, lo podemos tratar como un activo individual.

> Por otra parte, vimos que si conocemos los rendimientos esperados de cada activo que conforma el portafolio $E[r_i]$, podemos calcular el rendimiento esperado del portafolio como el promedio ponderado de los rendimientos esperados de los activos

$E[r_p]=\sum_{i=1}^{n}w_iE[r_i]$.

> Sin embargo, vimos que esto no es válido para la medida de riesgo (desviación estándar). Es decir, la varianza (o volatilidad, o desviación estándar) no es el promedio ponderado de las varianzas individuales. Anticipamos que esto es clave en el concepto de **diversificación**.

**Objetivos:**
- Medir el riesgo en un portafolio a partir del riesgo de cada uno de los activos que lo conforman.


*Referencias*
- Notas del curso "Portfolio Selection and Risk Management", Rice University, disponible en Coursera.
___

## 1. Midiendo el riesgo en un portafolio

### 1.1. Volatilidad de un portafolio

Retomamos el ejemplo qur veníamos trabajando la clase pasada...

**Ejemplo.** Supongamos que tenemos inversión en activos de Toyota, Walmart y Pfizer. Tenemos cuatro posibles estados económicos:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Creamos tabla
tabla = pd.DataFrame(columns=['Prob', 'Toyota', 'Walmart', 'Pfizer'], index=['Expansion', 'Normal', 'Recesion'])
tabla.index.name = 'Estado'
tabla['Prob']=np.array([0.3, 0.4, 0.3])
tabla['Toyota']=np.array([-.20, 0.05, 0.4])
tabla['Walmart']=np.array([-0.05, 0.10, 0.15])
tabla['Pfizer']=np.array([0.05, 0.03, 0.02])

tabla.round(4)

In [ ]:
# Incluimos una fila para rendimientos esperados
tabla.loc['Rend_Esp']=np.array([None, np.sum(tabla.Prob*tabla.Toyota),
                               np.sum(tabla.Prob*tabla.Walmart),
                               np.sum(tabla.Prob*tabla.Pfizer)
                               ])
tabla.round(4)

In [79]:
## Volatilidad DESV EST -> raíz de varianza (STANDARD DEV)
            #(en el caso de [:4] se indica que es tomar los 1ero 4)
# Toyota
stdT=np.sqrt(np.sum(tabla.Prob[:4]*(tabla.Toyota[:4]-tabla.Toyota[4])**2))
# Walmart
stdW=np.sqrt(np.sum(tabla.Prob[:4]*(tabla.Walmart[:4]-tabla.Walmart[4])**2))
# Pfizer
stdP=np.sqrt(np.sum(tabla.Prob[:4]*(tabla.Pfizer[:4]-tabla.Pfizer[4])**2))

stdT, stdW, stdP


(0.040174618853201331, 0.024088378940891809, 0.051110175112202463)

In [15]:
# Portafolio 0.5Toyota+0.5Pfizer
tabla['Port_TP']=0.5*tabla.Toyota+0.5*tabla.Pfizer
tabla.round(4)

,Prob,Toyota,Walmart,Pfizer,Port_TP
Estado,,,,,
Expansion,0.1,0.060,0.0450,0.0250,0.0425
Normal,0.4,0.075,0.0550,-0.0050,0.0350
Recesion,0.3,0.020,0.0400,0.0100,0.0150
Depresion,0.2,-0.030,-0.0100,0.1300,0.0500
Rend_Esp,NaN,0.036,0.0365,0.0295,0.0328


In [18]:
# Volatilidad del portafolio
stdPort_TP= stdT=np.sqrt(np.sum(tabla.Prob[:4]*(tabla.Port_TP[:4]-tabla.Port_TP[4])**2))
stdPort_TP

0.012867109232457772

**Actividad.** Encontrar la volatilidad del portafolio formado $0.5$ Toyota y $0.5$ Walmart.

In [25]:
# Encontrar los rendimientos del portafolio en cada estado de la economía
tabla['Port_TW']=0.5*tabla.Toyota+0.5*tabla.Walmart
tabla.round(4)

,Prob,Toyota,Walmart,Pfizer,Port_TP,Port_TW
Estado,,,,,,
Expansion,0.1,0.060,0.0450,0.0250,0.0425,0.0525
Normal,0.4,0.075,0.0550,-0.0050,0.0350,0.0650
Recesion,0.3,0.020,0.0400,0.0100,0.0150,0.0300
Depresion,0.2,-0.030,-0.0100,0.1300,0.0500,-0.0200
Rend_Esp,NaN,0.036,0.0365,0.0295,0.0328,0.0363


In [30]:
# Encontrar el rendimiento esperado del portafolio
    # Se da solo con la tabla el cáluclo del rend. esp.
tabla.Port_TW[-1]

0.036250000000000004

In [22]:
# Encontrar la volatilidad de Toyota, Walmart y el portafolio
stdPort_TW= stdT=np.sqrt(np.sum(tabla.Prob[:4]*(tabla.Port_TW[:4]-tabla.Port_TW[4])**2))
stdPort_TW

0.031647472252930409

### 1.2. Midiendo el co-movimiento entre instrumentos

- Una vez más, concluimos que la volatilidad (varianza) **NO** es el promedio ponderado de las varianzas individales.

- Por el contrario, la varianza de los rendimientos de un portafolio está afectada por el movimiento relativo de un activo individual respecto a otro.

- Por tanto, necesitamos definir las medidas de **covarianza** y **correlación**, que nos permiten evaluar las fluctuaciones relativas entre los activos.

#### Covarianza:

Es una medida el movimiento relativo entre dos instrumentos.

Matemáticamente, si tenemos dos activos $A_1$ y $A_2$ cuyos rendimientos son $r_1$ y $r_2$, respectivamente, entonces la covarianza de los rendimientos de los activos es

$$\text{cov}(r_1,r_2)=\sigma_{12}=\sigma_{21}=\sum_{j=1}^{m}p_j(r_{1j}-E[r_1])(r_{2j}-E[r_2]).$$

Podemos notar fácilmente que la covarianza de los rendimientos de un activo con los rendimientos del mismo activo corresponde a la varianza

$$\text{cov}(r_1,r_1)=\sigma_{11}=\sum_{j=1}^{m} p_j(r_{1j}-E[r_1])(r_{1j}-E[r_1])=\sigma_1^2=\text{var}(r_1).$$

**Ejemplo.** Calcular la covarianza entre los rendimientos de Toyota y Pfizer.

In [31]:
# Mostrar tabla
tabla.round(4)

,Prob,Toyota,Walmart,Pfizer,Port_TP,Port_TW
Estado,,,,,,
Expansion,0.1,0.060,0.0450,0.0250,0.0425,0.0525
Normal,0.4,0.075,0.0550,-0.0050,0.0350,0.0650
Recesion,0.3,0.020,0.0400,0.0100,0.0150,0.0300
Depresion,0.2,-0.030,-0.0100,0.1300,0.0500,-0.0200
Rend_Esp,NaN,0.036,0.0365,0.0295,0.0328,0.0363


In [76]:
# Calcular la covarianza
cov_TP= np.sum(tabla.Prob[:4]*(tabla.Toyota[:4]-tabla.Toyota[4])
                             *(tabla.Pfizer[:4]-tabla.Pfizer[4]))
cov_TP

-0.0017820000000000004

**Actividad.** Calcular la covarianza entre los rendimientos de Toyota y Walmart.

In [77]:
# Mostrar tabla
tabla.round(4)

,Prob,Toyota,Walmart,Pfizer,Port_TP,Port_TW
Estado,,,,,,
Expansion,0.1,0.060,0.0450,0.0250,0.0425,0.0525
Normal,0.4,0.075,0.0550,-0.0050,0.0350,0.0650
Recesion,0.3,0.020,0.0400,0.0100,0.0150,0.0300
Depresion,0.2,-0.030,-0.0100,0.1300,0.0500,-0.0200
Rend_Esp,NaN,0.036,0.0365,0.0295,0.0328,0.0363


In [78]:
# Calcular la covarianza
cov_TW= np.sum(tabla.Prob[:4]*(tabla.Toyota[:4]-tabla.Toyota[4])
                             *(tabla.Walmart[:4]-tabla.Walmart[4]))
cov_TW

0.0009060000000000001

**¿Qué nos dice este número?**
- El signo nos dice la dirección relativa entre los rendimientos de cada activo. Por ejemplo, la covarianza entre los rendimientos de Toyota y Pfizer es negativa... ver rendimientos.
- La magnitud de la covarianza no dice mucho acerca de la fuerza de relación estos rendimientos. 

**Correlación**
Un posible problema de la covarianza es que la magnitud de esta medida no nos dice mucho acerca de la fuerza de los movimientos relativos. La *correlación* es una medida normalizada del movimiento relativo entre los rendimientos de dos activos.

Matemáticamente,

$$\text{corr}(r_1,r_2)=\rho_{12}=\rho_{21}=\frac{\sigma_{12}}{\sigma_1\sigma_{2}}.$$

Propiedades:

- Podemos notar fácilmente que la correlación de los rendimientos de un activo con los rendimientos del mismo activo es $1$: $$\text{corr}(r_1,r_1)=\rho_{11}=\frac{\sigma_{11}}{\sigma_1\sigma_1}=\frac{\sigma_{1}^2}{\sigma_1\sigma_1}=1.$$
- El signo de la correlación y la covarianza es el mismo.
- La correlación satisface: $$-1\leq\rho_{12}\leq 1.$$

**Ejemplo.** Calcular la correlación entre los rendimientos de Toyota y Pfizer.

In [80]:
corr_TP = cov_TP/(stdT*stdP)
corr_TP

-0.86785778896609023

**Actividad.** Calcular la correlación entre los rendimientos de Toyota y Walmart.

In [81]:
corr_TW = cov_TW/(stdT*stdW)
corr_TW

0.93620047668219752

**Conclusión.**
- Es una medida normalizada de la fluctuación relativa de los rendimientos de dos activos. 
- En los ejemplos vistos, sería conveniente invertir en el Portafolio T-P puesto que su correlación es negativa. Por lo tanto, impacta menos un movimiento súbito al otro, por lo tanto es mejor para diversificar riesgo. 
___

## 2. Uniendo todo...

- Entonces, vimos mediante ejemplos que el riesgo en un portafolio se ve afectado significativamente por como los rendimientos de los activos se mueven relativamente.
- Este movimiento relativo lo medimos mediante la covarianza o la correlación.
- Si se mueven de una manera que no están perfectamente correlacionados ($\rho<1$), entonces el riesgo del portafolio siempre será menor que el promedio ponderado de los riesgos individuales.

<img style="float: left; margin: 0px 0px 15px 15px;" src="https://www.publicdomainpictures.net/pictures/20000/velka/happy-child.jpg" width="300px" height="200px" />

## Ésta es la razón por la que combinar activos en un portafolio permite diversificar el riesgo...

Entonces, ¿cómo podemos incorporar esta medida en el cálculo de la varianza del portafolio? 
- <font color=blue> Ver en el tablero...</font>
- ¿Cómo sería para dos activos?

**Ejemplo.** Calcular por fórmula para el portafolio de Toyota y Pfizer. Comparar.

w2*Desvest2+w2*desvest2+2*w*w*desvest(ww)

In [82]:
stdPort_TP2 = np.sqrt((0.5*stdT)**2+(0.5*stdP)**2+2*0.5*0.5*cov_TP)
stdPort_TP2

0.012867109232457763

**Actividad.** Calcular por fórmula para el portafolio de Toyota y Walmart. Comparar.

In [83]:
stdPort_TW2 = np.sqrt((0.5*stdT)**2+(0.5*stdW)**2+2*0.5*0.5*cov_TW)
stdPort_TW2

0.031647472252930409

## 2.1. <font color=blue> Ver en el tablero...</font>
### Matriz de varianza covarianza.
### Matriz de correlación.

# Anuncios parroquiales

## 1. Recordar quiz la siguiente clase. Temas: Clases 7 y 8.
## 2. Tarea: revisar archivo "Tarea4_MidiendoRendimientoRiesgo" en clase.
## 3. A más tardar el miércoles antes de clase tengo la realimentación de la tarea 3.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>